In [29]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

import statistics

from scipy.stats import rv_continuous
from scipy.stats import *

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance


######### Instance Transfer repositories ####################
from adapt.instance_based import TwoStageTrAdaBoostR2

print("Repositories uploaded!!")

Repositories uploaded!!


In [30]:
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Second Upload Completed!!")

Second Upload Completed!!


In [3]:
########################################## Larger Dataset (US) ###############################################################

US_df = pd.read_csv('US_Monthly_2011.csv')
# US_df = US_df.sort_values(by = ['rid'])
# print("Count values before the drop: ")
# print(US_df['rid'].value_counts())
# print(US_df.isnull().sum().sum())
US_df = US_df.dropna()
# print("Count values after the drop: ")
# print(US_df['rid'].value_counts())


Source_US_df = US_df.loc[US_df['rid'].isin(['3', '4', '5', '9'])]
Target_US_df = US_df.loc[US_df['rid'].isin(['1','2','6','7','8'])]

train_Target_US_df = Target_US_df.loc[Target_US_df['rid'].isin(['2','7','8'])]
test_Target_US_df = Target_US_df.loc[Target_US_df['rid'].isin(['1','6'])]

drop_rid = ['rid']
Source_US_df = Source_US_df.drop(drop_rid, axis =1)
train_Target_US_df = train_Target_US_df.drop(drop_rid, axis =1)
test_Target_US_df = test_Target_US_df.drop(drop_rid, axis =1)

target_column = ['pm25_value']
Source_US_df_y = Source_US_df[target_column]
Source_US_df_X = Source_US_df.drop(target_column, axis = 1)

train_Target_US_df_y = train_Target_US_df[target_column]
train_Target_US_df_X = train_Target_US_df.drop(target_column, axis = 1)

test_Target_US_df_y = test_Target_US_df[target_column]
test_Target_US_df_X = test_Target_US_df.drop(target_column, axis = 1)

TF_train_X = pd.concat([Source_US_df_X, train_Target_US_df_X], sort= False)
TF_train_y = pd.concat([Source_US_df_y, train_Target_US_df_y], sort= False)

np_TF_train_X = TF_train_X.to_numpy()
np_TF_train_y = TF_train_y.to_numpy()
np_test_Target_US_df_X = test_Target_US_df_X.to_numpy()
np_test_Target_US_df_y = test_Target_US_df_y.to_numpy()

np_TF_train_y_list = np_TF_train_y.ravel()
np_test_Target_US_df_y_list = np_test_Target_US_df_y.ravel()

print(Source_US_df_X.shape)
print(train_Target_US_df_X.shape)

(5353, 62)
(2881, 62)


In [31]:
########################################## Shorter Individual Datasets (US) ###############################################################

US_df_train = pd.read_csv('US_data/US_10_slice/Train/Train_1.csv')
US_train_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'Latitude', 'Longitude', 'year', 'month', 'rid', 'clust']
US_df_train = US_df_train.drop(US_train_droplist, axis = 1)
US_df_train = US_df_train.rename(columns={"is": "ISS"})
print(US_df_train.shape)

US_df_transfer = pd.read_csv('US_data/US_10_slice/Transfer/Transfer_1.csv')
US_transfer_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'Latitude', 'Longitude', 'year', 'month', 'rid', 'clust']
US_df_transfer = US_df_transfer.drop(US_transfer_droplist, axis = 1)
US_df_transfer = US_df_transfer.rename(columns={"is": "ISS"})
print(US_df_transfer.shape)

US_df_test = pd.read_csv('US_data/US_10_slice/Test/Test_1.csv')
US_test_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'Latitude', 'Longitude', 'year', 'month', 'rid']
US_df_test = US_df_test.drop(US_test_droplist, axis = 1)
US_df_test = US_df_test.rename(columns={"is": "ISS"})
print(US_df_test.shape)
print(US_df_test.columns)

target_column = ['pm25_value']
US_df_train_y = US_df_train[target_column]
US_df_train_X = US_df_train.drop(target_column, axis = 1)

US_df_transfer_y = US_df_transfer[target_column]
US_df_transfer_X = US_df_transfer.drop(target_column, axis = 1)

US_df_test_y = US_df_test[target_column]
US_df_test_X = US_df_test.drop(target_column, axis = 1)

TF_train_X = pd.concat([US_df_transfer_X, US_df_train_X], sort= False)
TF_train_y = pd.concat([US_df_transfer_y, US_df_train_y], sort= False)

np_TF_train_X = TF_train_X.to_numpy()
np_TF_train_y = TF_train_y.to_numpy()

# np_TF_train_X = US_df_train_X.to_numpy()
# np_TF_train_y = US_df_train_y.to_numpy()

np_US_df_test_X = US_df_test_X.to_numpy()
np_US_df_test_y = US_df_test_y.to_numpy()

np_TF_train_y_list = np_TF_train_y.ravel()
np_US_df_test_y_list = np_US_df_test_y.ravel()


(145, 19)
(2974, 19)
(1318, 19)
Index(['pm25_value', 'narr_dpt', 'narr_vis', 'nldas_pevapsfc',
       'nldas_dlwrfsfc', 'nldas_dswrfsfc', 'nldas_cape', 'nldas_pressfc',
       'nldas_tmp2m', 'nldas_rh2m', 'nldas_ugrd10m', 'nldas_vgrd10m',
       'forest_cover', 'elev', 'emissi11', 'local', 'ISS', 'pd', 'gc_aod'],
      dtype='object')


In [16]:
######################################## Phase 1: Seeding Technique (US) ###################################################

kmeans = KMeans(n_clusters = 15, random_state=0).fit(US_df_transfer)

US_alternate_df = US_df_transfer.copy()
US_alternate_df_np = US_df_transfer.to_numpy()

idxlist = []
US_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in US_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", kinematics_alternate_df_np.shape)
    US_new_df_list.append(rowval)
    US_alternate_df = np.delete(US_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


US_new_df = pd.DataFrame(np.vstack(US_new_df_list))

print("Shape of dataset extracted: ")
print(US_new_df.shape)
print("----------------------------------------------")

Shape of dataset extracted: 
(15, 19)
----------------------------------------------


In [17]:
##################################################### Phase 2: Seeding (US) ################################################

US_alternate_transfer_df = US_df_transfer[1:].copy()
US_alternate_transfer_df_np = US_alternate_transfer_df.to_numpy()

idxlist2 = []
US_final_df_list = []

for row_nm in US_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in US_alternate_transfer_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", Elevators_alternate_source_df_np.shape)
    US_final_df_list.append(row_val)
    US_alternate_transfer_df_np = np.delete(US_alternate_transfer_df_np, row_idx, 0)
    idxlist2.append(row_idx)


US_final_df = pd.DataFrame(np.vstack(US_final_df_list), columns = US_df_transfer.columns)

print("----------------------------------------------")
print("Shape of source before :",US_df_transfer.shape)
US_df_transfer = pd.DataFrame(np.vstack(US_alternate_transfer_df_np), columns= US_df_transfer.columns)
print("Shape of source after :", US_df_transfer.shape)

print("----------------------------------------------")
print("Shape of target before :", US_df_train.shape)
US_df_train = pd.concat([US_df_train, US_final_df], ignore_index=True)
print("Shape of target after :", US_df_train.shape)

print("----------------------------------------------")

----------------------------------------------
Shape of source before : (3173, 19)
Shape of source after : (3157, 19)
----------------------------------------------
Shape of target before : (151, 19)
Shape of target after : (166, 19)
----------------------------------------------


In [18]:
################################## Finding best instances from the source dataset (US) ######################################################

US_df_transfer["ManDis"] = ""

US_df_train_mean = []
prow = US_df_train.mean()
US_df_train_mean = [prow.pm25_value, prow.narr_dpt, prow.narr_vis, prow.nldas_pevapsfc, prow.nldas_dlwrfsfc, prow.nldas_dswrfsfc, 
                          prow.nldas_cape, prow.nldas_pressfc, prow.nldas_tmp2m, prow.nldas_rh2m, prow.nldas_ugrd10m, prow.nldas_vgrd10m,
                          prow.forest_cover, prow.elev, prow.emissi11, prow.local, prow.ISS, prow.pd, prow.gc_aod]

rowidx = 0

for row in US_df_transfer.itertuples():
    row_list =[row.pm25_value, row.narr_dpt, row.narr_vis, row.nldas_pevapsfc, row.nldas_dlwrfsfc, row.nldas_dswrfsfc, 
            row.nldas_cape, row.nldas_pressfc, row.nldas_tmp2m, row.nldas_rh2m, row.nldas_ugrd10m, row.nldas_vgrd10m,
            row.forest_cover, row.elev, row.emissi11, row.local, row.ISS, row.pd, row.gc_aod]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = US_df_train_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    US_df_transfer.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

US_df_transfer = US_df_transfer.sort_values(by =['ManDis'])
US_df_transfer = US_df_transfer.head(884) 
US_df_transfer = US_df_transfer.drop(['ManDis'], axis =1)
US_df_transfer = US_df_transfer.reset_index(drop=True)

#################### Splitting with small target set and large source and test set #############
print("Target Set: ", US_df_train.shape)
print("Source Set: ", US_df_transfer.shape)
print("Test Set: ", US_df_test.shape)


target_column = ['pm25_value']
US_df_train_y = US_df_train[target_column]
US_df_train_X = US_df_train.drop(target_column, axis = 1)

US_df_transfer_y = US_df_transfer[target_column]
US_df_transfer_X = US_df_transfer.drop(target_column, axis = 1)

US_df_test_y = US_df_test[target_column]
US_df_test_X = US_df_test.drop(target_column, axis = 1)

TF_train_X = pd.concat([US_df_transfer_X, US_df_train_X], sort= False)
TF_train_y = pd.concat([US_df_transfer_y, US_df_train_y], sort= False)

np_TF_train_X = TF_train_X.to_numpy()
np_TF_train_y = TF_train_y.to_numpy()

# np_TF_train_X = US_df_train_X.to_numpy()
# np_TF_train_y = US_df_train_y.to_numpy()

np_US_df_test_X = US_df_test_X.to_numpy()
np_US_df_test_y = US_df_test_y.to_numpy()

np_TF_train_y_list = np_TF_train_y.ravel()
np_US_df_test_y_list = np_US_df_test_y.ravel()

print("---------------------------")

###############################################################################################################################################

Target Set:  (166, 19)
Source Set:  (884, 19)
Test Set:  (1312, 19)
---------------------------


In [19]:
####################### STrAdaBoost.R2 #########################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

# sample_size = [len(US_df_train_X), len(US_df_transfer_X)]
sample_size = [166, 884]

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

print("STrAdaBoost.R2")
print("-------------------------------------------")

r2scorelist_stradaboost_us = []
rmselist_stradaboost_us = []

for x in range(0, 50):

    model_stradaboost_us = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                          n_estimators = n_estimators, sample_size = sample_size,
                          steps = steps, fold = fold,
                          random_state = random_state)

    model_stradaboost_us.fit(np_TF_train_X, np_TF_train_y_list)
    y_pred_us = model_stradaboost_us.predict(np_US_df_test_X)

    mse_stradaboost_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_us))
    rmselist_stradaboost_us.append(mse_stradaboost_us)

    r2_score_stradaboost_us = pearsonr(np_US_df_test_y_list, y_pred_us)
    r2_score_stradaboost_us = (r2_score_stradaboost_us[0])**2
    r2scorelist_stradaboost_us.append(r2_score_stradaboost_us)
    
    
print("RMSE of StrAdaboost:", statistics.mean(rmselist_stradaboost_us))
print("R^2 of StrAdaboost:", statistics.mean(r2scorelist_stradaboost_us))
print("\n")
print("RMSE of StrAdaboost:", rmselist_stradaboost_us)
print("R^2 of StrAdaboost:", r2scorelist_stradaboost_us)


print("-------------------------------------------")

STrAdaBoost.R2
-------------------------------------------
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoost.R2
Inside STrAdaBoos

In [12]:
####################### GBR #########################################################
print("Gradient Boosting Regression")
print("-------------------------------------------")

r2scorelist_GBRTL_us = []
rmselist_GBRTL_us = []

for x in range(0, 50):

    model_GBRTL_us = GradientBoostingRegressor(learning_rate=0.01, max_depth = 4, n_estimators=1000, subsample=0.5)
    model_GBRTL_us.fit(np_TF_train_X, np_TF_train_y_list)


    y_pred_GBRTL_us = model_GBRTL_us.predict(np_US_df_test_X) 

    rmse_GBRTL_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_GBRTL_us))
    rmselist_GBRTL_us.append(rmse_GBRTL_us)
    
    r2_score_GBRTL_us = pearsonr(np_US_df_test_y_list, y_pred_GBRTL_us)
    r2_score_GBRTL_us = (r2_score_GBRTL_us[0])**2
    r2scorelist_GBRTL_us.append(r2_score_GBRTL_us)
    
print("RMSE of GBR:", statistics.mean(rmselist_GBRTL_us))
print("R^2 of GBR:", statistics.mean(r2scorelist_GBRTL_us))
print("\n")
print("RMSE of GBR:", rmselist_GBRTL_us)
print("R^2 of GBR:", r2scorelist_GBRTL_us)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------
RMSE of GBR: 5.56251097099219
R^2 of GBR: 0.2776362339389524


RMSE of GBR: [5.5411597443739575, 5.561288114807914, 5.527738881448332, 5.552548161509634, 5.5493137977480655, 5.572211775678991, 5.561297351080517, 5.557060029102334, 5.5523594109985375, 5.52699000936518, 5.5038332018683, 5.571279259200452, 5.568875780209655, 5.571803744000682, 5.551821348140438, 5.587477857280663, 5.545590610287125, 5.558498376177656, 5.597089444319093, 5.55993296227395, 5.56425054000915, 5.538300037323368, 5.58881565152048, 5.533448244485991, 5.519722280238179, 5.579828899735532, 5.558334092049247, 5.567376474716695, 5.551825701277512, 5.569004157789179, 5.542372548817128, 5.606160855245383, 5.599395529538694, 5.6045393170418585, 5.54975181872947, 5.590494851985524, 5.589363854773772, 5.581281946071844, 5.566563195252052, 5.562548392616455, 5.592407265186975, 5.564729555742541, 5.5703652241581345, 5.59414638928847, 5.553896115093008

In [28]:
####################### AdaBoost TL #########################################################
print("Adaptive Boosting Transfer Regression")
print("-------------------------------------------")

r2scorelist_Ada_us = []
rmselist_Ada_us = []

for x in range(0, 50):
    
    model_Ada_us = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_Ada_us.fit(np_TF_train_X, np_TF_train_y_list)

    y_pred_Ada_us = model_Ada_us.predict(np_US_df_test_X)
    
    rmse_Ada_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_Ada_us))
    rmselist_Ada_us.append(rmse_Ada_us)
    
    r2_score_Ada_us = pearsonr(np_US_df_test_y_list, y_pred_Ada_us)
    r2_score_Ada_us = (r2_score_Ada_us[0])**2
    r2scorelist_Ada_us.append(r2_score_Ada_us)
    
    
print("RMSE of Ada TL:", statistics.mean(rmselist_Ada_us))
print("R^2 of Ada TL:", statistics.mean(r2scorelist_Ada_us))
print("\n")
print("RMSE of Ada TL:", rmselist_Ada_us)
print("R^2 of Ada TL:", r2scorelist_Ada_us)

print("-------------------------------------------")

Adaptive Boosting Transfer Regression
-------------------------------------------
RMSE of Ada TL: 5.2777368833543346
R^2 of Ada TL: 0.3324581752255384


RMSE of Ada TL: [5.255130515486572, 5.27081335008089, 5.274764403720441, 5.319898762716226, 5.205408230199599, 5.259719983897834, 5.326503801442614, 5.386794839064668, 5.253393059081979, 5.360406165673547, 5.330557429488022, 5.266629256437549, 5.163925221790993, 5.339822148731938, 5.272244247187067, 5.201120385414515, 5.315961789657094, 5.293076030004561, 5.281100344717172, 5.354497033925497, 5.252661578566778, 5.231931454172448, 5.3066631018238555, 5.348414965950147, 5.277014266349626, 5.271994095321574, 5.272305388819804, 5.298905401150821, 5.337158629399414, 5.269564247646058, 5.304870023518099, 5.33271671201986, 5.262602783141486, 5.2617412781343225, 5.295191700602214, 5.2677985646119945, 5.231499150051405, 5.228412658041968, 5.278809658373867, 5.257555878664432, 5.284324581994705, 5.32881508125111, 5.209284640596778, 5.20275158635